This notebook builds a surrogate model for the macroeconomic impacts of improvements in biological aging.

To run this notebooks, use the `dynviz-dev` virtual environment, which can be build using the `environment.yml` file located in the same directory as this notebook.

In [1]:
# imports
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split


In [2]:
# Read in training data
df = pd.read_csv('SL_TrainingData.csv')
# Split the X and y variables
X = df[["r", "age", "start_year", "num_years", "mortality", "fertility", "productivity"]].values
y = df[["NPV_100", "GDP_20"]].values

In [3]:
# train-test split for model evaluation
X_train, X_test, y_train, y_test = train_test_split(
    X, y, train_size=0.7, shuffle=True
)

In [4]:
# Layer setting
num_input = X.shape[1]
num_hidden1 = 10 * num_input
num_hidden2 = 10 * num_input
num_output = y.shape[1]
layers_dim = [num_input, num_hidden1, num_hidden2, num_output]
print("Dimensions of each layer are {}".format(layers_dim))

Dimensions of each layer are [7, 70, 70, 2]


In [5]:
# Define the neural network
# We use [Keras](https://www.tensorflow.org/guide/keras) to define the
# neural network
nn = tf.keras.Sequential(
    [
        keras.layers.Dense(
            num_hidden1, activation="relu", input_shape=(num_input,)
        ),
        keras.layers.Dense(num_hidden2, activation="relu"),
        keras.layers.Dense(num_output),
    ]
)
print(nn.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 70)                560       
                                                                 
 dense_1 (Dense)             (None, 70)                4970      
                                                                 
 dense_2 (Dense)             (None, 2)                 142       
                                                                 
Total params: 5672 (22.16 KB)
Trainable params: 5672 (22.16 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None


In [6]:
loss_fn = tf.keras.losses.MeanSquaredError()
nn.compile(optimizer='adam',
              loss=loss_fn,
              metrics=['accuracy'])

In [7]:
nn.fit(X_train, y_train, epochs=200)

Epoch 1/200
354/354 [==============================] - 0s 588us/step - loss: 3.0303 - accuracy: 0.6658
Epoch 2/200
354/354 [==============================] - 0s 574us/step - loss: 0.2840 - accuracy: 0.7344
Epoch 3/200
354/354 [==============================] - 0s 565us/step - loss: 0.1696 - accuracy: 0.7376
Epoch 4/200
354/354 [==============================] - 0s 633us/step - loss: 0.1155 - accuracy: 0.7302
Epoch 5/200
354/354 [==============================] - 0s 588us/step - loss: 0.0786 - accuracy: 0.7330
Epoch 6/200
354/354 [==============================] - 0s 564us/step - loss: 0.0509 - accuracy: 0.7349
Epoch 7/200
354/354 [==============================] - 0s 590us/step - loss: 0.0328 - accuracy: 0.7382
Epoch 8/200
354/354 [==============================] - 0s 616us/step - loss: 0.0194 - accuracy: 0.7439
Epoch 9/200
354/354 [==============================] - 0s 657us/step - loss: 0.0157 - accuracy: 0.7428
Epoch 10/200
354/354 [==============================] - 0s 722us/step - l

In [8]:
nn.evaluate(X_test,  y_test, verbose=2)

152/152 - 0s - loss: 0.0015 - accuracy: 0.9998 - 112ms/epoch - 738us/step


[0.00154090765863657, 0.9997937679290771]

In [9]:
# save full model - not just weights
tf.keras.Model.save(nn, "SL_model_full.h5", save_format="h5")

/Users/jason.debacker/anaconda3/envs/dynviz-dev/lib/python3.11/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [10]:
# !tensorflowjs_converter --input_format=keras SL_model.h5 ./tf_model/
# !tensorflowjs_converter --input_format=tf_saved_model SL_model2 ./tf_model/
!tensorflowjs_converter --input_format=keras SL_model_full.h5 ./tf_model/

In [11]:
# Read model back in
model = keras.models.load_model("SL_model_full.h5")

In [12]:
# use model to make predictions
predictions = model.predict(X_test)
print(predictions)

152/152 [==============================] - 0s 375us/step
[[ 1.7426349  3.4481854]
 [ 3.428185   6.838882 ]
 [ 7.4028916  7.3531713]
 ...
 [ 6.3719816  6.3448586]
 [ 5.0916653 10.149454 ]
 [ 2.6199934  2.600796 ]]


In [13]:
# use model to make a single prediction
x = np.array([[2.89386640e-02, 4.00000000e+01, 1.00000000e+01, 2.00000000e+01,
       2.76861914e-01, 5.76141576e+00, 6.13201966e+00]])
print(model.predict(x)[0]) # Print the first (only) result

1/1 [==============================] - 0s 11ms/step
[4.8056154 4.786179 ]


In [14]:
!conda list tensorflow

# packages in environment at /Users/jason.debacker/anaconda3/envs/dynviz-dev:
#
# Name                    Version                   Build  Channel
tensorflow                2.15.0                   pypi_0    pypi
tensorflow-decision-forests 1.8.1                    pypi_0    pypi
tensorflow-estimator      2.15.0                   pypi_0    pypi
tensorflow-hub            0.16.1                   pypi_0    pypi
tensorflow-io-gcs-filesystem 0.37.1                   pypi_0    pypi
tensorflow-macos          2.15.0                   pypi_0    pypi
tensorflowjs              4.21.0                   pypi_0    pypi
